In [7]:
from elasticsearch import Elasticsearch
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

In [8]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)

session = tf.Session()
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

In [9]:
client = Elasticsearch()


In [22]:
INDEX_NAME = "images"
SEARCH_SIZE = 5
from IPython.display import HTML, display, update_display
from functools import reduce

def search(text):
    vectors = session.run(embeddings, feed_dict={text_ph: [text]})
    query_vector = [vector.tolist() for vector in vectors]
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, doc['title_vector']) + 1.0",
                "params": {"query_vector": query_vector[0]}
            }
        }
    }
    response = client.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source": {"includes": ["imageId", "title", "body"]}
        }
    )

    images = map(lambda hit: "<div style='margin: 10px;display:inline-block;vertical-align: top; height:250px; width: 300px'><img style='display:inline-block' src='https://images-cdn.kahoot.it/" + hit["_source"]["imageId"] + "?&width=300&crop=3:2'><br>" + hit["_source"]["title"] + "</div>", response["hits"]["hits"])

    return HTML(reduce(lambda x,y: x + y , images))
             
             

In [26]:
#display(search("who was the first man on the moon?"))
from ipywidgets import widgets
text = widgets.Text()
display(text)

display(HTML("<div/>"), display_id='results')
def doSearch(input):
    update_display(search(input.value), display_id='results');


text.on_submit(doSearch)

Text(value='')